# **LP1_Run3**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, a 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm.  """

In [ ]:


response1 = openai.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:
C1 = Available volume of chemical component C1 (10,000 l)
C2 = Available volume of chemical component C2 (15,000 l)
D_GCA = Demand for GCA (6,000 l)
D_GCB = Demand for GCB (7,000 l)
D_GCC = Demand for GCC (9,000 l)
P_GCA = Profit from selling 1 l of GCA ($120)
P_GCB = Profit from selling 1 l of GCB ($135)
P_GCC = Profit from selling 1 l of GCC ($155)
C1_ratio = Level of critical element in C1 (0.4)
C2_ratio = Level of critical element in C2 (0.2)
GCA_content = Proportional amount of the critical element required to make GCA (0.3)
GCB_content = Maximum proportional amount of the critical element allowed in GCB (0.3)
GCC_ratio = Minimum ratio of C1 to C2 in GCC production (0.3)

Decision Variables:
X_GCA = Volume of GCA to produce
X_GCB = Volume of GCB to produce
X_GCC = Volume of GCC to produce
Y1_GCA = Volume of C1 used for GCA 
Y2_GCA = Volume of C2 used for GCA
Y1_GCB = Volume of C1 used for GCB 
Y2_GCB = Volume of C2 used for GCB
Y1_GCC = Volume of C1 used for GCC 
Y2_GC

### **Generate Pyomo Code**

In [ ]:

response2 = openai.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the Pyomo code for the problem defined above:

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Initialize parameters
model.C1 = Param(initialize=10000)
model.C2 = Param(initialize=15000)
model.D_GCA = Param(initialize=6000)
model.D_GCB = Param(initialize=7000)
model.D_GCC = Param(initialize=9000)
model.P_GCA = Param(initialize=120)
model.P_GCB = Param(initialize=135)
model.P_GCC = Param(initialize=155)
model.C1_ratio = Param(initialize=0.4)
model.C2_ratio = Param(initialize=0.2)
model.GCA_content = Param(initialize=0.3)
model.GCB_content = Param(initialize=0.3)
model.GCC_ratio = Param(initialize=0.3)

# Initialize decision variables
model.X_GCA = Var(domain=NonNegativeReals)
model.X_GCB = Var(domain=NonNegativeReals)
model.X_GCC = Var(domain=NonNegativeReals)
model.Y1_GCA = Var(domain=NonNegativeReals)
model.Y2_GCA = Var(domain=NonNegativeReals)
model.Y1_GCB = Var(domain=NonNegativeReals)
model.Y2_GCB = Var(domain=NonNegativeReals)
model.Y1_GCC = Var(domain=NonNegativeReals)
model.Y2_GCC = Var(domain=NonNegativeReals)

# ObjectiveFunction
def obj_expression(model): 
    return model.P_GCA*model.X_GCA + model.P_GCB*model.X_GCB + model.P_GCC*model.X_GCC
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# Constraints
def c1_rule(model):
    return model.Y1_GCA + model.Y1_GCB + model.Y1_GCC <= model.C1
model.Constraint1 = Constraint(rule=c1_rule)

def c2_rule(model):
    return model.Y2_GCA + model.Y2_GCB + model.Y2_GCC <= model.C2
model.Constraint2 = Constraint(rule=c2_rule)

def c3_rule(model):
    return model.X_GCA >= model.D_GCA
model.Constraint3 = Constraint(rule=c3_rule)

def c4_rule(model):
    return model.X_GCB >= model.D_GCB
model.Constraint4 = Constraint(rule=c4_rule)

def c5_rule(model):
    return model.X_GCC >= model.D_GCC
model.Constraint5 = Constraint(rule=c5_rule)

def c6_rule(model):
    return model.X_GCA == model.Y1_GCA + model.Y2_GCA
model.Constraint6 = Constraint(rule=c6_rule)

def c7_rule(model):
    return model.X_GCB == model.Y1_GCB + model.Y2_GCB
model.Constraint7 = Constraint(rule=c7_rule)

def c8_rule(model):
    return model.X_GCC == model.Y1_GCC + model.Y2_GCC
model.Constraint8 = Constraint(rule=c8_rule)

def c9_rule(model):
    return model.Y1_GCA*model.C1_ratio / model.X_GCA >= model.GCA_content
model.Constraint9 = Constraint(rule=c9_rule)

def c10_rule(model):
    return (model.Y1_GCB*model.C1_ratio + model.Y2_GCB*model.C2_ratio) / model.X_GCB <= model.GCB_content
model.Constraint10 = Constraint(rule=c10_rule)

def c11_rule(model):
    return model.Y1_GCC / model.X_GCC >= model.GCC_ratio
model.Constraint11 = Constraint(rule=c11_rule)

# Solve
SolverFactory('glpk').solve(model)

# print results
print('X_GCA:', model.X_GCA())
print('X_GCB:', model.X_GCB())
print('X_GCC:', model.X_GCC())
```

In this code, all the sample data is encapsulated within the parameter initializations. The model, decision variables, constraints and objective function are all formulated according to the problem description. Note that the decision variables are bound from below by zero because we cannot produce a negative volume of the products. The solver used here is 'glpk', because it can handle linear problems like this one.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Initialize parameters
model.C1 = Param(initialize=10000)
model.C2 = Param(initialize=15000)
model.D_GCA = Param(initialize=6000)
model.D_GCB = Param(initialize=7000)
model.D_GCC = Param(initialize=9000)
model.P_GCA = Param(initialize=120)
model.P_GCB = Param(initialize=135)
model.P_GCC = Param(initialize=155)
model.C1_ratio = Param(initialize=0.4)
model.C2_ratio = Param(initialize=0.2)
model.GCA_content = Param(initialize=0.3)
model.GCB_content = Param(initialize=0.3)
model.GCC_ratio = Param(initialize=0.3)

# Initialize decision variables
model.X_GCA = Var(domain=NonNegativeReals)
model.X_GCB = Var(domain=NonNegativeReals)
model.X_GCC = Var(domain=NonNegativeReals)
model.Y1_GCA = Var(domain=NonNegativeReals)
model.Y2_GCA = Var(domain=NonNegativeReals)
model.Y1_GCB = Var(domain=NonNegativeReals)
model.Y2_GCB = Var(domain=NonNegativeReals)
model.Y1_GCC = Var(domain=NonNegativeReals)
model.Y2_GCC = Var(domain=NonNegativeReals)

# ObjectiveFunction
def obj_expression(model):
    return model.P_GCA*model.X_GCA + model.P_GCB*model.X_GCB + model.P_GCC*model.X_GCC
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# Constraints
def c1_rule(model):
    return model.Y1_GCA + model.Y1_GCB + model.Y1_GCC <= model.C1
model.Constraint1 = Constraint(rule=c1_rule)

def c2_rule(model):
    return model.Y2_GCA + model.Y2_GCB + model.Y2_GCC <= model.C2
model.Constraint2 = Constraint(rule=c2_rule)

def c3_rule(model):
    return model.X_GCA >= model.D_GCA
model.Constraint3 = Constraint(rule=c3_rule)

def c4_rule(model):
    return model.X_GCB >= model.D_GCB
model.Constraint4 = Constraint(rule=c4_rule)

def c5_rule(model):
    return model.X_GCC >= model.D_GCC
model.Constraint5 = Constraint(rule=c5_rule)

def c6_rule(model):
    return model.X_GCA == model.Y1_GCA + model.Y2_GCA
model.Constraint6 = Constraint(rule=c6_rule)

def c7_rule(model):
    return model.X_GCB == model.Y1_GCB + model.Y2_GCB
model.Constraint7 = Constraint(rule=c7_rule)

def c8_rule(model):
    return model.X_GCC == model.Y1_GCC + model.Y2_GCC
model.Constraint8 = Constraint(rule=c8_rule)

def c9_rule(model):
    return model.Y1_GCA*model.C1_ratio / model.X_GCA >= model.GCA_content
model.Constraint9 = Constraint(rule=c9_rule)

def c10_rule(model):
    return (model.Y1_GCB*model.C1_ratio + model.Y2_GCB*model.C2_ratio) / model.X_GCB <= model.GCB_content
model.Constraint10 = Constraint(rule=c10_rule)

def c11_rule(model):
    return model.Y1_GCC / model.X_GCC >= model.GCC_ratio
model.Constraint11 = Constraint(rule=c11_rule)

# Solve
SolverFactory('glpk').solve(model)

# print results
print('X_GCA:', model.X_GCA())
print('X_GCB:', model.X_GCB())
print('X_GCC:', model.X_GCC())

ValueError: Model constraint (Constraint9) contains nonlinear terms that cannot be written to LP format

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Initialize parameters
model.C1 = Param(initialize=10000)
model.C2 = Param(initialize=15000)
model.D_GCA = Param(initialize=6000)
model.D_GCB = Param(initialize=7000)
model.D_GCC = Param(initialize=9000)
model.P_GCA = Param(initialize=120)
model.P_GCB = Param(initialize=135)
model.P_GCC = Param(initialize=155)
model.C1_ratio = Param(initialize=0.4)
model.C2_ratio = Param(initialize=0.2)
model.GCA_content = Param(initialize=0.3)
model.GCB_content = Param(initialize=0.3)
model.GCC_ratio = Param(initialize=0.3)

# Initialize decision variables
model.X_GCA = Var(domain=NonNegativeReals)
model.X_GCB = Var(domain=NonNegativeReals)
model.X_GCC = Var(domain=NonNegativeReals)
model.Y1_GCA = Var(domain=NonNegativeReals)
model.Y2_GCA = Var(domain=NonNegativeReals)
model.Y1_GCB = Var(domain=NonNegativeReals)
model.Y2_GCB = Var(domain=NonNegativeReals)
model.Y1_GCC = Var(domain=NonNegativeReals)
model.Y2_GCC = Var(domain=NonNegativeReals)

# ObjectiveFunction
def obj_expression(model):
    return model.P_GCA*model.X_GCA + model.P_GCB*model.X_GCB + model.P_GCC*model.X_GCC
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# Constraints
def c1_rule(model):
    return model.Y1_GCA + model.Y1_GCB + model.Y1_GCC <= model.C1
model.Constraint1 = Constraint(rule=c1_rule)

def c2_rule(model):
    return model.Y2_GCA + model.Y2_GCB + model.Y2_GCC <= model.C2
model.Constraint2 = Constraint(rule=c2_rule)

def c3_rule(model):
    return model.X_GCA >= model.D_GCA
model.Constraint3 = Constraint(rule=c3_rule)

def c4_rule(model):
    return model.X_GCB >= model.D_GCB
model.Constraint4 = Constraint(rule=c4_rule)

def c5_rule(model):
    return model.X_GCC >= model.D_GCC
model.Constraint5 = Constraint(rule=c5_rule)

def c6_rule(model):
    return model.X_GCA == model.Y1_GCA + model.Y2_GCA
model.Constraint6 = Constraint(rule=c6_rule)

def c7_rule(model):
    return model.X_GCB == model.Y1_GCB + model.Y2_GCB
model.Constraint7 = Constraint(rule=c7_rule)

def c8_rule(model):
    return model.X_GCC == model.Y1_GCC + model.Y2_GCC
model.Constraint8 = Constraint(rule=c8_rule)

def c9_rule(model):
    return model.Y1_GCA*model.C1_ratio >= model.GCA_content*model.X_GCA
model.Constraint9 = Constraint(rule=c9_rule)

def c10_rule(model):
    return (model.Y1_GCB*model.C1_ratio + model.Y2_GCB*model.C2_ratio) <= model.GCB_content*model.X_GCB
model.Constraint10 = Constraint(rule=c10_rule)

def c11_rule(model):
    return model.Y1_GCC >= model.GCC_ratio*model.X_GCC
model.Constraint11 = Constraint(rule=c11_rule)

# Solve
SolverFactory('glpk').solve(model)

# print results
print('X_GCA:', model.X_GCA())
print('Y1_GCA', model.Y1_GCA())
print('Y2_GCA', model.Y2_GCA())
print('X_GCB:', model.X_GCB())
print('Y1_GCB', model.Y1_GCB())
print('Y2_GCB', model.Y2_GCB())
print('X_GCC:', model.X_GCC())
print('Y1_GCC', model.Y1_GCC())
print('Y2_GCC', model.Y2_GCC())

print('Profit', model.OBJ())

X_GCA: 6000.0
Y1_GCA 6000.0
Y2_GCA 0.0
X_GCB: 7000.0
Y1_GCB 400.0
Y2_GCB 6600.0
X_GCC: 12000.0
Y1_GCC 3600.0
Y2_GCC 8400.0
Profit 3525000.0
